In [62]:
import cvxpy as cp
import numpy as np

In [63]:
# Input Data
channels = ['Social Media', 'SEM', 'Content', 'Email', 'Display']
expected_returns = [28, 34, 21, 14, 18]  # per $100
min_allocations = [5000, 5000, 2500, 2500, 2500]  # Minimum allocations
max_allocations = [20000, 20000, 15000, 10000, 10000]  # Maximum allocations
total_budget = 50000  # Total budget
target_acquisitions = 13825  # Target acquisitions

# Convert expected returns per $100 to per dollar
returns_per_dollar = np.array(expected_returns) / 100

In [64]:
# Assumed covariance matrix (example from previous discussions)
cov_matrix = np.array([
    [0.0002, 0.00015, 0.0001, 0.00005, 0.00005],  # Social Media
    [0.00015, 0.0003, 0.0001, 0.00005, 0.00005],  # SEM
    [0.0001, 0.0001, 0.0002, 0.00005, 0.00005],   # Content
    [0.00005, 0.00005, 0.00005, 0.0001, 0.00002], # Email
    [0.00005, 0.00005, 0.00005, 0.00002, 0.0001]  # Display
])


In [65]:
# Compute H matrix for the quadratic objective function
H = 2 * cov_matrix

# Decision variables: allocations for each channel
x = cp.Variable(len(channels))

# Objective function: minimize portfolio variance (1/2 x^T H x)
objective = cp.Minimize((1/2) * cp.quad_form(x, H))

In [66]:
# Constraints
constraints = [
    cp.sum(x) == total_budget,  # Total budget constraint
    returns_per_dollar @ x >= target_acquisitions,  # Minimum expected acquisitions
    x >= min_allocations,  # Minimum allocation constraints
    x <= max_allocations,  # Maximum allocation constraints
    x >= 0  # Non-negativity constraint
]

In [67]:
# Solve the problem
problem = cp.Problem(objective, constraints)
problem.solve()

344549.99999999965

In [68]:
# Summary of the Model
def model_summary():
    print("\n--- Quadratic Programming Model Summary ---")
    print(f"Status: {problem.status}")
    print(f"Optimal Value (Minimized Risk): {problem.value:.6f}")
    print(f"Total Expected Acquisitions: {returns_per_dollar @ x.value:.2f}")
    print("\nChannel-wise Allocations and Expected Acquisitions:")
    for i, channel in enumerate(channels):
        allocation = x.value[i]
        acquisitions = allocation * returns_per_dollar[i]
        print(f"{channel}:")
        print(f"  Allocation = ${allocation:.2f}")
        print(f"  Expected Acquisitions = {acquisitions:.2f}")
    print("\nConstraints Validation:")
    print(f"  Total Allocated Budget: {sum(x.value):.2f} (Should equal {total_budget})")
    print(f"  Total Expected Acquisitions: {returns_per_dollar @ x.value:.2f} (Should be >= {target_acquisitions})")
    for i, channel in enumerate(channels):
        print(f"  {channel}:")
        print(f"    Allocation >= {min_allocations[i]}: {x.value[i] >= min_allocations[i]}")
        print(f"    Allocation <= {max_allocations[i]}: {x.value[i] <= max_allocations[i]}")
    print("\nCovariance Matrix:")
    print(cov_matrix)
    print("---------------------------------------------")

In [69]:
# Display the model summary
model_summary()


--- Quadratic Programming Model Summary ---
Status: optimal
Optimal Value (Minimized Risk): 344550.000000
Total Expected Acquisitions: 13825.00

Channel-wise Allocations and Expected Acquisitions:
Social Media:
  Allocation = $16500.00
  Expected Acquisitions = 4620.00
SEM:
  Allocation = $20000.00
  Expected Acquisitions = 6800.00
Content:
  Allocation = $2500.00
  Expected Acquisitions = 525.00
Email:
  Allocation = $2500.00
  Expected Acquisitions = 350.00
Display:
  Allocation = $8500.00
  Expected Acquisitions = 1530.00

Constraints Validation:
  Total Allocated Budget: 50000.00 (Should equal 50000)
  Total Expected Acquisitions: 13825.00 (Should be >= 13825)
  Social Media:
    Allocation >= 5000: True
    Allocation <= 20000: True
  SEM:
    Allocation >= 5000: True
    Allocation <= 20000: True
  Content:
    Allocation >= 2500: True
    Allocation <= 15000: True
  Email:
    Allocation >= 2500: True
    Allocation <= 10000: True
  Display:
    Allocation >= 2500: True
    All

In [70]:
# Results
print("Optimal Budget Allocation and Acquisitions:")
for i, channel in enumerate(channels):
    allocation = x.value[i]
    acquisitions = allocation * returns_per_dollar[i]
    print(f"{channel}: Allocation = ${allocation:.2f}, Expected Acquisitions = {acquisitions:.2f}")


Optimal Budget Allocation and Acquisitions:
Social Media: Allocation = $16500.00, Expected Acquisitions = 4620.00
SEM: Allocation = $20000.00, Expected Acquisitions = 6800.00
Content: Allocation = $2500.00, Expected Acquisitions = 525.00
Email: Allocation = $2500.00, Expected Acquisitions = 350.00
Display: Allocation = $8500.00, Expected Acquisitions = 1530.00


In [71]:
print(f"\nTotal Expected Acquisitions: {returns_per_dollar @ x.value:.2f}")
print(f"Minimum Total Variance (Risk): {problem.value:.6f}")


Total Expected Acquisitions: 13825.00
Minimum Total Variance (Risk): 344550.000000
